In [170]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
%matplotlib inline
from sklearn.preprocessing import StandardScaler, LabelEncoder
import pickle

import seaborn as sns

In [171]:
df = pd.read_csv('../data/raw/train.csv')
print("The data is read.")

The data is read.


In [172]:
df.shape

(1460, 81)

In [173]:
df.head()

,Id,MSSubClass,MSZoning,LotFrontage,LotArea,Street,Alley,LotShape,LandContour,Utilities,...,PoolArea,PoolQC,Fence,MiscFeature,MiscVal,MoSold,YrSold,SaleType,SaleCondition,SalePrice
0,1,60,RL,65.0,8450,Pave,NaN,Reg,Lvl,AllPub,...,0,NaN,NaN,NaN,0,2,2008,WD,Normal,208500
1,2,20,RL,80.0,9600,Pave,NaN,Reg,Lvl,AllPub,...,0,NaN,NaN,NaN,0,5,2007,WD,Normal,181500
2,3,60,RL,68.0,11250,Pave,NaN,IR1,Lvl,AllPub,...,0,NaN,NaN,NaN,0,9,2008,WD,Normal,223500
3,4,70,RL,60.0,9550,Pave,NaN,IR1,Lvl,AllPub,...,0,NaN,NaN,NaN,0,2,2006,WD,Abnorml,140000
4,5,60,RL,84.0,14260,Pave,NaN,IR1,Lvl,AllPub,...,0,NaN,NaN,NaN,0,12,2008,WD,Normal,250000


## Removing outliers

In [174]:
def feature_encode(df):
    # categorical feature encoding
    cat_columns = df.select_dtypes(include = ['object']).columns.to_list()
    cat_encoder = {}
    for i in cat_columns:
        en = LabelEncoder()
        df[i] = en.fit_transform(df[i])
        cat_encoder[i] = en
    
    file = open('../models/data-cleaning-models/cat_encoder.pkl', 'wb')
    pickle.dump(cat_encoder, file)
    return df

In [175]:
def null_columns(df):
    # null values
    to_remove = []
    null_val = df.isnull().sum()
    null_val = null_val[null_val.values > df.shape[0]*0.2]
    to_remove = np.append(to_remove, null_val.index.to_list())
    return to_remove

In [176]:
def poor_corr_columns(df):
    # poor correlation from numeric data
    to_remove = []
    num_df = df.select_dtypes(include = [np.number])
    corr_mat = num_df.corr()
    corr_mat = corr_mat['SalePrice'].sort_values(ascending = False)
    poor_corr = corr_mat[(corr_mat.values < 0.35) & (corr_mat.values > -0.035)]
    to_remove = np.append(to_remove, poor_corr.index.to_list())
    return to_remove

In [177]:
def data_scale(df):
    numeric_col = df.select_dtypes(include = [np.number]).columns.to_list()
    num_col_x = numeric_col[:-1]
    num_col_y = numeric_col[-1]
    scaler_x = StandardScaler()
    scaled_x_df = scaler_x.fit_transform(df[num_col_x])
    scaler_y = StandardScaler()
    scaled_y_df = scaler_y.fit_transform(df[num_col_y].values.reshape(-1, 1))
    
    file1 = open('../models/data-cleaning-models/scaler_x.pkl', 'wb')
    pickle.dump(scaler_x, file1)
    
    file2 = open('../models/data-cleaning-models/scaler_y.pkl', 'wb')
    pickle.dump(scaler_y, file2)
    
    return num_col_x, num_col_y, scaled_x_df, scaled_y_df

In [178]:
def clean_train(df):
    
    # Removing the outliers from 
    Q1 = df.SalePrice.quantile(0.25)
    Q3 = df.SalePrice.quantile(0.75)
    IQR = Q3 - Q1
    bound = Q3 + 3 * IQR
    df.drop(df[df.SalePrice > bound].index, axis = 0, inplace = True)
    
    # multicollinearity
    to_remove = np.array(['Id','GarageCars','1stFlrSF','GrLivArea','FullBath'], dtype = object) # Observation from data visualization
    
    #poor correlation
    to_remove = np.append(to_remove, poor_corr_columns(df))
    
    # null handling
    to_remove = np.append(to_remove, null_columns(df))
    
    # Print to remove
    print(to_remove)
    
    # removing columns
    df.drop(to_remove, axis = 1, inplace = True)
    
    # Dropping null values
    df.dropna(inplace = True)
    
    # # categorical feature encoding
    # df = feature_encode(df)
    
    return df, to_remove.tolist()

In [179]:
df, to_remove = clean_train(df)

Poor corr:
['LotFrontage' 'WoodDeckSF' 'OpenPorchSF' '2ndFlrSF' 'HalfBath' 'LotArea'
 'BsmtFullBath' 'BsmtUnfSF' 'BedroomAbvGr' 'ScreenPorch' 'MoSold'
 '3SsnPorch' 'PoolArea' 'BsmtFinSF2' 'MiscVal' 'BsmtHalfBath' 'YrSold'
 'Id']
['Id' 'GarageCars' '1stFlrSF' 'GrLivArea' 'FullBath' 'LotFrontage'
 'WoodDeckSF' 'OpenPorchSF' '2ndFlrSF' 'HalfBath' 'LotArea' 'BsmtFullBath'
 'BsmtUnfSF' 'BedroomAbvGr' 'ScreenPorch' 'MoSold' '3SsnPorch' 'PoolArea'
 'BsmtFinSF2' 'MiscVal' 'BsmtHalfBath' 'YrSold' 'Id' 'Alley' 'FireplaceQu'
 'PoolQC' 'Fence' 'MiscFeature']


In [180]:
df.head()

,MSSubClass,MSZoning,Street,LotShape,LandContour,Utilities,LotConfig,LandSlope,Neighborhood,Condition1,...,GarageYrBlt,GarageFinish,GarageArea,GarageQual,GarageCond,PavedDrive,EnclosedPorch,SaleType,SaleCondition,SalePrice
0,60,RL,Pave,Reg,Lvl,AllPub,Inside,Gtl,CollgCr,Norm,...,2003.0,RFn,548,TA,TA,Y,0,WD,Normal,208500
1,20,RL,Pave,Reg,Lvl,AllPub,FR2,Gtl,Veenker,Feedr,...,1976.0,RFn,460,TA,TA,Y,0,WD,Normal,181500
2,60,RL,Pave,IR1,Lvl,AllPub,Inside,Gtl,CollgCr,Norm,...,2001.0,RFn,608,TA,TA,Y,0,WD,Normal,223500
3,70,RL,Pave,IR1,Lvl,AllPub,Corner,Gtl,Crawfor,Norm,...,1998.0,Unf,642,TA,TA,Y,272,WD,Abnorml,140000
4,60,RL,Pave,IR1,Lvl,AllPub,FR2,Gtl,NoRidge,Norm,...,2000.0,RFn,836,TA,TA,Y,0,WD,Normal,250000


In [181]:
num_col_x,num_col_y, scaled_x_df, scaled_y_df = data_scale(df)
df[num_col_x],df[num_col_y] = scaled_x_df,scaled_y_df

In [182]:
df.head()

,MSSubClass,MSZoning,Street,LotShape,LandContour,Utilities,LotConfig,LandSlope,Neighborhood,Condition1,...,GarageYrBlt,GarageFinish,GarageArea,GarageQual,GarageCond,PavedDrive,EnclosedPorch,SaleType,SaleCondition,SalePrice
0,0.091599,RL,Pave,Reg,Lvl,AllPub,Inside,Gtl,CollgCr,Norm,...,0.994239,RFn,0.269575,TA,TA,Y,-0.351385,WD,Normal,0.363437
1,-0.875250,RL,Pave,Reg,Lvl,AllPub,FR2,Gtl,Veenker,Feedr,...,-0.096083,RFn,-0.208054,TA,TA,Y,-0.351385,WD,Normal,-0.025192
2,0.091599,RL,Pave,IR1,Lvl,AllPub,Inside,Gtl,CollgCr,Norm,...,0.913475,RFn,0.595231,TA,TA,Y,-0.351385,WD,Normal,0.579341
3,0.333311,RL,Pave,IR1,Lvl,AllPub,Corner,Gtl,Crawfor,Norm,...,0.792328,Unf,0.779769,TA,TA,Y,4.103103,WD,Abnorml,-0.622528
4,0.091599,RL,Pave,IR1,Lvl,AllPub,FR2,Gtl,NoRidge,Norm,...,0.873092,RFn,1.832724,TA,TA,Y,-0.351385,WD,Normal,0.960773


In [183]:
# categorical feature encoding
df = feature_encode(df)

In [184]:
df.head()

,MSSubClass,MSZoning,Street,LotShape,LandContour,Utilities,LotConfig,LandSlope,Neighborhood,Condition1,...,GarageYrBlt,GarageFinish,GarageArea,GarageQual,GarageCond,PavedDrive,EnclosedPorch,SaleType,SaleCondition,SalePrice
0,0.091599,3,1,3,3,0,4,0,5,2,...,0.994239,1,0.269575,4,4,2,-0.351385,8,4,0.363437
1,-0.875250,3,1,3,3,0,2,0,24,1,...,-0.096083,1,-0.208054,4,4,2,-0.351385,8,4,-0.025192
2,0.091599,3,1,0,3,0,4,0,5,2,...,0.913475,1,0.595231,4,4,2,-0.351385,8,4,0.579341
3,0.333311,3,1,0,3,0,0,0,6,2,...,0.792328,2,0.779769,4,4,2,4.103103,8,0,-0.622528
4,0.091599,3,1,0,3,0,2,0,15,2,...,0.873092,1,1.832724,4,4,2,-0.351385,8,4,0.960773


In [185]:
df.to_csv("../data/processed/clean_train_data.csv", index = False)

## Test dataset

In [186]:
def test_encode(df_test):
    cat_columns = df_test.select_dtypes(include = ['object']).columns.to_list()
    picklefile_enc = open('../models/data-cleaning-models/cat_encoder.pkl', 'rb')
    cat_encoder = pickle.load(picklefile_enc)
    
    for i in cat_columns:
        encoder = cat_encoder[i]
        df_test[i]= encoder.fit_transform(df_test[i])
        
    return df_test

In [187]:
def test_scale(df_test):
    picklefile_scale = open('../models/data-cleaning-models/scaler_x.pkl', 'rb')
    scaler_x = pickle.load(picklefile_scale)
    
    numeric_col = df_test.select_dtypes(include = [np.number]).columns.to_list()
    scaled_x_df_test = scaler_x.fit_transform(df_test[num_col_x])
    df_test[num_col_x]= scaled_x_df_test
    
    return df_test

In [188]:
def clean_test(df_test, to_remove):
    df_test.drop(to_remove, axis = 1, inplace = True)
    
    df_test = test_scale(df_test)
    
    df_test.dropna(inplace = True)
    
    df_test = test_encode(df_test)
        
    return df_test

In [189]:
df_test = pd.read_csv('../data/raw/test.csv')
print("The data is read.")

The data is read.


In [190]:
len(to_remove)

28

In [191]:
df_test.head()

,Id,MSSubClass,MSZoning,LotFrontage,LotArea,Street,Alley,LotShape,LandContour,Utilities,...,ScreenPorch,PoolArea,PoolQC,Fence,MiscFeature,MiscVal,MoSold,YrSold,SaleType,SaleCondition
0,1461,20,RH,80.0,11622,Pave,NaN,Reg,Lvl,AllPub,...,120,0,NaN,MnPrv,NaN,0,6,2010,WD,Normal
1,1462,20,RL,81.0,14267,Pave,NaN,IR1,Lvl,AllPub,...,0,0,NaN,NaN,Gar2,12500,6,2010,WD,Normal
2,1463,60,RL,74.0,13830,Pave,NaN,IR1,Lvl,AllPub,...,0,0,NaN,MnPrv,NaN,0,3,2010,WD,Normal
3,1464,60,RL,78.0,9978,Pave,NaN,IR1,Lvl,AllPub,...,0,0,NaN,NaN,NaN,0,6,2010,WD,Normal
4,1465,120,RL,43.0,5005,Pave,NaN,IR1,HLS,AllPub,...,144,0,NaN,NaN,NaN,0,1,2010,WD,Normal


In [192]:
df_test_clean = clean_test(df_test, to_remove)

In [194]:
df_test_clean.head()

,MSSubClass,MSZoning,Street,LotShape,LandContour,Utilities,LotConfig,LandSlope,Neighborhood,Condition1,...,GarageType,GarageYrBlt,GarageFinish,GarageArea,GarageQual,GarageCond,PavedDrive,EnclosedPorch,SaleType,SaleCondition
0,-0.874711,2,1,3,3,0,4,0,12,1,...,1,-0.632862,2,1.185538,3,4,2,-0.360738,8,4
1,-0.874711,3,1,0,3,0,0,0,12,2,...,1,-0.746405,2,-0.740959,3,4,2,-0.360738,8,4
2,0.061351,3,1,0,3,0,4,0,8,2,...,1,0.729660,0,0.042545,3,4,2,-0.360738,8,4
3,0.061351,3,1,0,3,0,4,0,8,2,...,1,0.767508,0,-0.012761,3,4,2,-0.360738,8,4
4,1.465443,3,1,0,1,0,4,0,22,2,...,1,0.540421,1,0.153157,3,4,2,-0.360738,8,4


In [195]:
df_test.to_csv('../data/processed/clean_test_data.csv',index = False)